In [ ]:
import hpmcm
import tables_io
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATADIR = "shear_tract"
shear_st = "0p01"
SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_wmom_{shear_st}_cleaned_*.pq")))
SOURCE_TABLEFILES.reverse()
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

REF_DIR = (37.91729, 6.94389)  # RA, DEC in deg
REGION_SIZE = (1.2, 1.2)  # in Deg
#CELL_SIZE = 5.0e-5    # in Deg
CELL_SIZE = 1. / (3600*2) # in Deg = 0.5"
#SUBREGION_SIZE = 2700 # in Pixels
SUBREGION_SIZE = 1350 # in Pixels
PIXEL_R2CUT = 1.

In [ ]:
SOURCE_TABLEFILES

In [ ]:
matcher = hpmcm.Match.create(REF_DIR, REGION_SIZE, CELL_SIZE, pixelR2Cut=PIXEL_R2CUT, subRegionSize=SUBREGION_SIZE)

In [ ]:
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

In [ ]:
matcher._redData[0]

In [ ]:
oDict = matcher.analyzeCell(2, 2, True)

In [ ]:
cellData = oDict['cellData']
cd = cellData._clusterDict
all_clusters = list(cd.keys())
idx = list(cd.keys())[0]
image = oDict['image']
cluster = cd[idx]


In [ ]:
def showObjects(img, cluster, cellData):
    extent = (0, cluster._footprint.getBBox().getWidth(),
              0, cluster._footprint.getBBox().getHeight())
    cluster.extract(cellData)
    xOffset = cellData._minPix[0] + cluster._footprint.getBBox().getBeginY()
    yOffset = cellData._minPix[1] + cluster._footprint.getBBox().getBeginX()
    xOff = cluster.xCell - xOffset
    yOff = cluster.yCell - yOffset
    img = plt.imshow(image[cluster._footprint.getBBox()].array, origin='lower', extent=extent)
    cb = plt.colorbar()
    colors = ['red', 'blue', 'green', 'cyan', 'orange']
    for iObj, obj in enumerate(cluster.objects):
        xC = obj._xCent - xOffset
        yC = obj._yCent - yOffset
        img.axes.scatter(yOff[obj._mask], xOff[obj._mask], c=colors[iObj%5])
        img.axes.scatter(yC, xC, marker='+', c=colors[iObj % 6])
    return img.axes.figure


In [ ]:
oDict['cellData'].data[0]

In [ ]:
_ = showObjects(image, cd[all_clusters[7]], cellData)

In [ ]:
cd[all_clusters[7]].data